In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.optimizers import Adam


# Data Preparation

In [2]:
def extract_faces(data_dir, output_dir, img_size=(128, 128)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    for cls in os.listdir(data_dir):
        class_path = os.path.join(data_dir, cls)
        if not os.path.isdir(class_path):
            continue

        output_class_path = os.path.join(output_dir, cls)
        os.makedirs(output_class_path, exist_ok=True)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                face = img[y:y + h, x:x + w]
                face = cv2.resize(face, img_size)
                cv2.imwrite(os.path.join(output_class_path, img_name), face)

In [3]:
DATA_DIR = ".\Data\Original Images\Original Images"
OUTPUT_DIR = "Data\CroppedData"

#extract_faces(DATA_DIR, OUTPUT_DIR)

In [4]:
# 3. Data Preparation
def prepare_data(data_dir, img_size=(128, 128), batch_size=4):
    datagen = ImageDataGenerator(
        validation_split=0.2,
        rescale=1.0/255,
        horizontal_flip=True,
        rotation_range=20,
        shear_range=10
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=2,
        class_mode='categorical',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=1,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, val_generator

# 4. EfficientNet Model
def create_efficientnet(input_shape, num_classes):
    base_model = EfficientNetB0(weights=None, input_shape=input_shape, include_top=False)
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 5. Custom CNN Model
def create_custom_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 6. ResNet Model
def create_resnet(input_shape, num_classes):
    base_model = ResNet50(weights=None, input_shape=input_shape, include_top=False, pooling='avg')
    model = Sequential([
        base_model,
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 7. Training and Evaluation
def train_and_evaluate(models, train_generator, val_generator):
    metrics = {}
    for model_name, model in models.items():
        print(f"Training {model_name}...")
        history = model.fit(train_generator, validation_data=val_generator, epochs=10)
        loss, acc = model.evaluate(val_generator, verbose=0)
        metrics[model_name] = acc
        print(f"{model_name} - Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    return metrics# 3. Data Preparation
def prepare_data(data_dir, img_size=(128, 128), batch_size=32):
    datagen = ImageDataGenerator(
        validation_split=0.2,
        rescale=1.0/255,
        horizontal_flip=True,
        rotation_range=20,
        shear_range=10
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, val_generator

# 4. EfficientNet Model
def create_efficientnet(input_shape, num_classes):
    base_model = EfficientNetB0(weights=None, input_shape=input_shape, include_top=False)
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 5. Custom CNN Model
def create_custom_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 6. ResNet Model
def create_resnet(input_shape, num_classes):
    base_model = ResNet50(weights=None, input_shape=input_shape, include_top=False, pooling='avg')
    model = Sequential([
        base_model,
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 7. Training and Evaluation
def train_and_evaluate(models, train_generator, val_generator):
    metrics = {}
    for model_name, model in models.items():
        print(f"Training {model_name}...")
        history = model.fit(train_generator, validation_data=val_generator, epochs=10)
        loss, acc = model.evaluate(val_generator, verbose=0)
        metrics[model_name] = acc
        print(f"{model_name} - Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    return metrics

In [ ]:
# Step 3: Data Preparation
train_gen, val_gen = prepare_data(OUTPUT_DIR)

# Step 4: Model Creation
input_shape = train_gen.image_shape
num_classes = train_gen.num_classes

models = {
    "EfficientNet": create_efficientnet(input_shape, num_classes),
    "CustomCNN": create_custom_cnn(input_shape, num_classes),
    "ResNet": create_resnet(input_shape, num_classes)
}

# Step 5: Train and Evaluate
metrics = train_and_evaluate(models, train_gen, val_gen)

# Identify Best Model
best_model_name = max(metrics, key=lambda x: metrics[x]['accuracy'])
print(f"Best Model: {best_model_name}")

Found 1579 images belonging to 29 classes.
Found 381 images belonging to 29 classes.
